<a href="https://colab.research.google.com/github/gabrielbruschi/predicao_acoes/blob/main/rede_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.0 MB/s 
     |████████████████████████████████| 9.8 MB 31.5 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('ggplot')

import pmdarima
import statsmodels.tsa.api as smt

import python_utils as ts

In [ ]:
df = pd.read_csv('PTR4_bovespa_tratado.csv')
df = df.set_index(pd.DatetimeIndex(df['data_pregao'].values))
df

,preco_abertura,preco_maximo,preco_minimo,preco_fechamento,qtd_negocios,volume_negocio,data_pregao
2002-01-01,6.39,6.39,6.39,6.39,3.186628,0.0,2002-01-01
2002-01-02,6.45,6.49,6.39,6.41,3.192857,23920000.0,2002-01-02
2002-01-03,6.49,6.50,6.42,6.45,3.214662,39043200.0,2002-01-03
2002-01-04,6.38,6.45,6.32,6.41,3.194726,21500800.0,2002-01-04
2002-01-07,6.38,6.60,6.38,6.55,3.264501,40406400.0,2002-01-07
...,...,...,...,...,...,...,...
2022-05-06,32.69,33.12,31.90,33.06,33.060001,129570200.0,2022-05-06
2022-05-09,32.70,33.04,32.06,32.16,32.160000,92314500.0,2022-05-09
2022-05-10,32.36,32.85,32.20,32.44,32.439999,60330600.0,2022-05-10
2022-05-11,32.78,33.91,32.69,33.57,33.570000,108895400.0,2022-05-11


In [ ]:
dtf = df[['data_pregao', 'preco_fechamento']]
dtf = dtf.rename(columns = {'data_pregao':'data', 'preco_fechamento':'x'})
df_acao = dtf.set_index(pd.DatetimeIndex(dtf['data'].values))
#df_acao.dropna(inplace=True)
df_acao

,data,x
2002-01-01,2002-01-01,6.39
2002-01-02,2002-01-02,6.41
2002-01-03,2002-01-03,6.45
2002-01-04,2002-01-04,6.41
2002-01-07,2002-01-07,6.55
...,...,...
2022-05-06,2022-05-06,33.06
2022-05-09,2022-05-09,32.16
2022-05-10,2022-05-10,32.44
2022-05-11,2022-05-11,33.57


In [ ]:
#Melhores parametros do ARIMA; order e seasonal_order
best_model = pmdarima.auto_arima(dtf['x'],                               
                                 seasonal=True, stationary=False, 
                                 m=7, information_criterion='aic', 
                                 max_order=20,                                     
                                 max_p=10, max_d=3, max_q=10,                                     
                                 max_P=10, max_D=3, max_Q=10,                                   
                                 error_action='ignore')
print("best model --> (p, d, q):", best_model.order, " and  (P, D, Q, s):", best_model.seasonal_order)

best model --> (p, d, q): (2, 1, 3)  and  (P, D, Q, s): (0, 0, 0, 7)


In [ ]:
print("best model --> (p, d, q):", best_model.order, " and  (P, D, Q, s):", best_model.seasonal_order)

best model --> (p, d, q): (2, 1, 3)  and  (P, D, Q, s): (0, 0, 0, 7)


In [ ]:
model = pmdarima.auto_arima(
  dtf['x'],
	trace=True,
	error_action="ignore",
	suppress_warnings=True)

print("best model --> (p, d, q):", model.order, " and  (P, D, Q, s):", model.seasonal_order)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=8444.135, Time=10.60 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=8445.830, Time=0.65 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=8447.816, Time=0.40 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=8447.816, Time=0.76 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=8444.310, Time=0.15 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=8449.972, Time=3.39 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=8449.828, Time=3.23 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=8429.160, Time=11.01 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=8443.842, Time=3.27 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=8430.786, Time=13.87 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=8430.384, Time=12.07 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=8429.087, Time=9.59 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=8443.303, Time=3.78 sec
 ARIMA(2,1,4)(0,0,0)[0] intercept   : AIC=8430.568, Time=13.29 sec
 ARIMA(1,1,4)(0,0,0)[0] inte

In [ ]:
print("best model --> (p, d, q):", model.order, " and  (P, D, Q, s):", model.seasonal_order)

best model --> (p, d, q): (2, 1, 3)  and  (P, D, Q, s): (0, 0, 0, 0)


In [ ]:
forecast = model.predict()
forecast

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


5077    33.882215
5078    33.756011
5079    33.808730
5080    33.876940
5081    33.773717
5082    33.798742
5083    33.869517
5084    33.788719
5085    33.792898
5086    33.861059
dtype: float64

In [ ]:
forecast_best = best_model.predict()
forecast_best

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  data=self.data,


5077    33.882215
5078    33.756011
5079    33.808730
5080    33.876940
5081    33.773717
5082    33.798742
5083    33.869517
5084    33.788719
5085    33.792898
5086    33.861059
dtype: float64